# Merging data frames

Merging data is a very common data wrangling task. We might find one variable in one dataset and another variable in some other dataset. The more observations we have, the harder it is to compile data from disparate sources by hand. The **tidyverse**, however, makes it fairly simple to pull columns of data together based on some identifying variable. 

Again, merging is a very common task, especially when we work with aggregate data or data on political units (counties, states, countries, etc). It can get tricky! You need to look at your data, understand which variables "define" your observations (e.g. uniquely identify the unit of analysis), and then make sure the variables are compatible. Remember to frequently check your data frames to make sure the merge is happening the way you expect it to. Let's get started!

In [ ]:
# install only if needed
# install.packages('pacman')
pacman::p_load(tidyverse)

For this lab, we're going to reuse a dataset on NJ school districts' graduation rates, along with some new data. We want to combine these data with two new data frames: one from the NJ Department of Education about teacher retention, and a second from the U.S. Census Bureau's American Community Survey with information about income and poverty rates in NJ counties.  

To do this, we'll take advantage of the `paste()` function from base R which allows us to combine two strings of text characters together. First, we store the beginning of the GitHub url as `part1` and then we combine that with the file name to make working urls that point to our various data frames.

In [ ]:
part1 <- "https://raw.githubusercontent.com/bowendc/pol200_labs/main/"

nj.grad <- read_csv(
                paste(part1, "grad_rates.csv", sep = "")
                )
nj.retention <- read_csv(
                paste(part1, "nj_retention.csv", sep = "")
                )
nj.poverty <- read_csv(
                paste(part1, "nj_county_poverty.csv", sep = "")
                )

Let's take a look at each of these data frames.

In [ ]:
head(nj.grad)
head(nj.retention)
head(nj.poverty)

Looks like `read_csv` is pulling in row names a variable in two of our data frames (probably due to how they were originally created and stored). Let's re-import these data using the `col_select` argument to avoid the columns starting with `...1`.

In [ ]:
nj.grad <- read_csv(
                paste(part1, "grad_rates.csv", sep = ""),
                col_select = !`...1`                         # again, ! means "not"
                )
nj.poverty <- read_csv(
                paste(part1, "nj_county_poverty.csv", sep = ""),
                col_select = !`...1`
                )

We also need to clean the data a little bit. For example, `nj.retention` currently has observations split into rows for Administrators and Teachers. We only want the data for teachers, so we need to filter out the administrator rows or we'll create duplicate observations when we merge. Second, there is an observation for State-wide retention. Let's drop that as well. 

In [ ]:
nj.retention <- nj.retention |> filter(`Teachers/Admins` == "Teachers" & DistrictCode != "State")

I typically use `left_join()` to merge data frames. `left_join()` is from the **dplyr** package, which is included in the **tidyverse**. It will match observations in the first listed data frame with observations listed in the second listed data frame. If an observation is unmatched in the first data frame, it will remain in the dataset. If it is missing in the second, it is dropped. `right_join` does the opposite (keeps unmatched observations from only the second data frame). `full_join` keeps unmatched observations from both data frames. Key to using any of these functions is understanding how the observations are supposed to be matched. 

You can define that using the `by` argument. If the variable is named the same thing in both data frames, you could simply use: `by = join_by(varname)` (obviously using the actual variable name).

In [ ]:
combined <- left_join(nj.grad, nj.retention,
                        by = join_by(District.Code == DistrictCode))

Hmmm. This is a common issue. What's going on is that one version of the numeric ID is being stored as a number, in the other data frame it was stored as a character (text) variable. We can quickly fix using `as.numeric()` to convert our variable to a number, and then we can try our merge again.

In [ ]:
nj.retention <- nj.retention |> mutate(DistrictCode = as.numeric(DistrictCode))
 
combined <- left_join(nj.grad, nj.retention,
                        by = join_by(District.Code == DistrictCode))

Nice! Now all of our district-level data are included in the same data frame. 

We can use the same functions to merge data from multiple levels. Always treat the lowest level of aggregation as your "main" data, then merge in higher-levels of data by observation. Here, we join our district data with county-level data on poverty using the county name variables included in our data frames.

In [ ]:
combined2 <- left_join(combined, nj.poverty, 
                        by = join_by(County.Name == County))

head(combined2)

Great! It looks like it worked. Notice that every district in the county as the same value for `pct.poverty`. That's because the poverty rate is measured at the county level, and we joined the data frames by county. 